# Modular AI Agent Demo – LangChain + Groq + Tools

##  What This Notebook Demonstrates

This notebook is a **minimal but functional demonstration of Modular AI Agents**, as discussed in the article _“How Modular AI Agents Work”_. It shows how a large language model (LLM) can be used not just to generate text, but to **intelligently orchestrate tasks** across multiple tools — like a smart project manager assigning jobs to specialists.

---

##  Why This Matters

In modern AI applications, it's not enough for a model to just answer questions. We want agents that can:
- Break down complex prompts into smaller tasks
- Choose the right tool for each task (e.g., a calculator or a search engine)
- Combine the results to respond intelligently

This type of orchestration is the core idea behind **Modular AI Agents**.

---

## How It Works in This Notebook

1. **The User Prompt**:  
   The user gives a compound query like:  
   `"Summarize who Marie Curie was and also compute 20 * 4"`

2. **The Agent**:  
   The LLM (Groq’s LLaMA3 in this case) receives this prompt and decides:
   - That “who Marie Curie was” requires a search → it uses the **Wikipedia tool**
   - That “compute 20 * 4” requires math → it uses the **Calculator tool**

3. **The Tools**:  
   - The **WikipediaAPIWrapper** fetches factual info from Wikipedia
   - A **custom Calculator function** evaluates basic math expressions

4. **The Orchestration**:  
   The agent invokes the tools in the right order, based on task type.  
   This is done using LangChain’s `initialize_agent()` function and `ZERO_SHOT_REACT_DESCRIPTION` agent type.

---

## What This Shows

This code proves — in a very small but clear way — how modular agents work:
- They are **task-aware**
- They use **tools like plugins**
- They **don’t hardcode logic**; the LLM dynamically plans and routes each query

---

## Tools & Libraries Used

- **LangChain** – for agent creation and tool abstraction  
- **Groq LLaMA3** – as the language model (fast inference)  
- **Wikipedia API** – as a fact retrieval tool  
- **Custom Calculator** – a simple math parser (using `eval()`)

---

 Scroll down to see the actual code and output of this modular agent in action.


##  Installing Required Libraries

Before we build our Modular AI Agent, we need to install some essential libraries that power this architecture:

- `langchain`, `langchain-community`, `langchain-core`  
    These are the core libraries for building agents, managing tools, and orchestrating logic flow.

- `wikipedia`  
    Used to fetch real-time factual data (e.g., summaries of people, places, events) from Wikipedia.

- `groq` + `langchain-groq`  
    Allows us to connect LangChain to the **Groq API**, which provides access to the ultra-fast **LLaMA3 model** for reasoning and planning.

These tools will enable our agent to **think**, **plan**, and **act** using real modules — just like a team of experts working behind the scenes.


In [ ]:
pip install langchain langchain-community langchain-core wikipedia groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain-groq


##  Building the Tool Modules

In Modular AI, each "tool" is a specialized skill — like memory, code execution, search, or math.

Here, we define two such **modular tools**:


### 1.  Calculator Module
We created a simple Python function that takes a math expression (like `45 ** 2`) and uses Python’s built-in `eval()` to compute it. This simulates a tool that handles **numeric reasoning**.

- Example Input: `"100 + 25"`
- Output: `"125"`

In [ ]:
def custom_calculator(input_text: str) -> str:
    try:
        result = eval(input_text)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


### 2.  Wikipedia Module
Using LangChain's built-in `WikipediaAPIWrapper`, this module lets the agent **search real-time information** on topics like countries, people, or historical events — directly from Wikipedia.

By wrapping each of these in a `Tool` object, we turn them into plug-and-play modules that the AI agent can call whenever needed.

Think of it like giving your AI different apps on its phone — one for calculations, another for looking things up on the web!

In [ ]:
# Wikipedia tool using LangChain's community wrapper
from langchain.agents import Tool
from langchain_community.utilities import WikipediaAPIWrapper

# Wikipedia Tool
wikipedia = WikipediaAPIWrapper()
wiki_tool = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="Searches Wikipedia for factual information about places, people, etc."
)

# Calculator Tool (custom)
calc_tool = Tool(
    name="Calculator",
    func=custom_calculator,
    description="Evaluates basic math expressions like '45 ** 2', '10 + 5', etc."
)


## The Orchestrator Agent

In Modular or Agentic AI, we don't rely on a single big model to do everything. Instead:

> We give the AI access to a **toolkit** — and let it decide *when* and *how* to use each tool.

Here’s what happens behind the scenes:

###  1. LLM as the Brain
We use `LLaMA3` (a modern open-source model) from **Groq**, known for high-speed inference. This acts as the **decision-maker**.

###  2. Plugging in Tools
We plug in two tools:
- **Wikipedia** for factual lookups
- **Calculator** for math logic

The LLM is not doing everything itself. Instead, it reasons like this:
- “Ah, I need to find the capital of Germany → call the Wikipedia tool.”
- “Now I need to solve 45 ** 2 → call the Calculator tool.”

###  3. The Agent is the Project Manager
LangChain’s agent system is like a **smart project manager**.
It:
- Breaks down user queries
- Decides which tool to use
- Passes the input to the right tool
- Collects results and returns the final answer

This entire process demonstrates **Modular AI orchestration** — where intelligence comes from **coordination between modules**, not just brute-force modeling.


In [ ]:
# Loading the language model from Groq (using LLaMA3)
from langchain.agents import initialize_agent, AgentType
from langchain_groq import ChatGroq

# Initialize the LLM
llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    api_key="your_groq_api_key"  # Replace with your actual API key
)

# List of tools we defined earlier
tools = [wiki_tool, calc_tool]

# Create the agent (Orchestrator)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=2 # Limit to prevent infinite loops
)


## Final Output

We gave a **multi-part instruction**:

> "What is the capital of Germany and what is 45 ** 2?"

Here’s how the Modular AI system handled it:

1.  **Used the Wikipedia tool** to search factual information about the capital of Germany.
2.  **Used the Calculator tool** to evaluate the math expression `45 ** 2` (which means 45 squared).
3.  **The agent coordinated everything** — it figured out which tool to use for which sub-task, executed them, and gave a combined answer.



In [ ]:
# Run a sample query through the agent
query = "What is the capital of Germany and what is 45 ** 2?"
response = agent.invoke({"input": query})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to answer two parts to this question. The first part is about the capital of Germany, which I can find using Wikipedia. The second part is about the calculation 45 ** 2, which I can evaluate using the Calculator.

Action: Wikipedia
Action Input: Berlin
Observation: Page: Berlin
Summary: Berlin ( bur-LIN; German: [bɛʁˈliːn] ) is the capital and largest city of Germany, by both area and population. With 3.7 million inhabitants, it has the highest population within its city limits of any city in the European Union. The city is also one of the states of Germany, being the third smallest state in the country by area. Berlin is surrounded by the state of Brandenburg, and Brandenburg's capital Potsdam is nearby. The urban area of Berlin has a population of over 4.6 million and is therefore the most populous urban area in Germany. The Berlin-Brandenburg capital region has around 6.2 million inhabitants and is Germany's second-largest metr

Let’s try another multi-step query:

> **"Summarize who Marie Curie was and also compute 20 * 4"**

This query combines:
-  A factual lookup (about Marie Curie) — handled by the **Wikipedia tool**.
-  A basic math expression (`20 * 4`) — handled by the **Calculator tool**.


#### Agent Behavior:

1. It first used the Wikipedia tool to fetch a brief summary of Marie Curie’s life and achievements.
2. Then it used the Calculator tool to compute the expression `20 * 4`, resulting in **80**.

Note: Since we limited the agent to **2 iterations**, it stopped before producing the final combined output — but both sub-tasks were handled correctly.


In [ ]:
query = "Summarize who Marie Curie was and also compute 20 * 4"
response = agent.invoke({"input": query})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to find information about Marie Curie and also perform a mathematical calculation.

Action: Wikipedia
Action Input: Marie Curie
Observation: Page: Marie Curie
Summary: Maria Salomea Skłodowska-Curie (Polish: [ˈmarja salɔˈmɛa skwɔˈdɔfska kʲiˈri] ; née Skłodowska; 7 November 1867 – 4 July 1934), known simply as Marie Curie ( KURE-ee; French: [maʁi kyʁi]), was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity. 
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields. Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes. She was, in 1906, the first woman to become a professor at the University of Paris.
She was born in Warsaw, in what was then the K

##  **Conclusion:** Modular AI Agents in Action

This simple project demonstrates the core idea of **Modular AI Agents** — combining independent tools like a **Wikipedia search** and a **Calculator** into a single intelligent workflow.

Even though the query "Summarize who Marie Curie was and also compute 20 * 4" may seem unusual, it highlights how an agent can coordinate multiple tasks using specialized modules.

###  Techniques Used:
- **LangChain's Agent Framework** (Zero-Shot ReAct)
- Tool Integration with `Tool()` for modularity
- LLM (LLaMA-3 via Groq) for reasoning and coordination


This approach is scalable and can be extended to build larger systems with dozens of tools — enabling flexible, interpretable, and reusable AI workflows.



###  Why This Is Important?

This notebook demonstrates the **core idea of Agentic AI**:
- It's not about building a massive monolithic model.
- It's about building **intelligent coordination** between small tools.
- The agent **plans, delegates, and executes** — just like a human manager would do with a team.

---

This is exactly what frameworks like **LangChain**, **AutoGen**, and **OpenAI’s Assistants API** aim to enable:  
> Agents that are smart, modular, and extensible.